# Mai24_CMLOPS_Accidents_Cdc_#2_Modeles

In [1]:
# Import des bibliothèques nécessaires au projet
import pandas as pd
import numpy as np
import warnings
import matplotlib.pyplot as plt
import seaborn as sns
#import plotly.express as px
#from plotly.offline import init_notebook_mode, iplot
import time

# Ignorer les avertissements
warnings.filterwarnings("ignore", category=pd.errors.DtypeWarning)

from imblearn.over_sampling import SMOTE
from sklearn.model_selection import train_test_split
from imblearn.under_sampling import RandomUnderSampler
from imblearn.metrics import classification_report_imbalanced
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score

from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
import pickle

In [2]:
#pip install imbalanced-learn

In [3]:
df = pd.read_csv('../src/data/data_2005a2021_final.csv', index_col=0)

In [4]:
df.head()
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 49998 entries, 201200035939 to 201200036992
Data columns (total 26 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   mois     49998 non-null  int64  
 1   jour     49998 non-null  int64  
 2   lum      49998 non-null  int64  
 3   agg      49998 non-null  int64  
 4   int      49998 non-null  int64  
 5   col      49998 non-null  float64
 6   com      49998 non-null  int64  
 7   dep      49998 non-null  int64  
 8   hr       49998 non-null  int64  
 9   mn       49998 non-null  int64  
 10  catv     49998 non-null  int64  
 11  choc     49998 non-null  float64
 12  manv     49998 non-null  float64
 13  place    49998 non-null  int64  
 14  catu     49998 non-null  int64  
 15  grav     49998 non-null  int64  
 16  trajet   49998 non-null  float64
 17  an_nais  49998 non-null  int64  
 18  catr     49998 non-null  int64  
 19  circ     49998 non-null  float64
 20  nbv      49998 non-null  int64  
 21 

In [7]:
df["grav"].value_counts()

grav
1    38316
3    10336
2     1346
Name: count, dtype: int64

## 1- Préparation du jeu de donnée - pré processing

In [11]:
# on sépare les variables cibles et les caractéristiques
X = df.drop(columns=['grav'])
y = df['grav']

In [12]:
# on divise les données en ensemble d'entraînement et de test
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [14]:
# nous avons vu dans notre analyse de données que les classes 2 et 3 étaient sous représentées
# afin de régler ce déséquilibre de classe, nous allons faire un SMOTE et l'appliquer à nos données
"""
smote = SMOTE(random_state=42)
X_train_resampled, y_train_resampled = smote.fit_resample(X_train, y_train)
"""

'\nsmote = SMOTE(random_state=42)\nX_train_resampled, y_train_resampled = smote.fit_resample(X_train, y_train)\n'

In [15]:
#print('Classes échantillon smote :', dict(pd.Series(y_train_resampled).value_counts()))
#le nombre de l'échantillon a augmenté (environ 4M)
#Classes échantillon smote : {1: 1335601, 3: 1335601, 2: 1335601}
#il vaut mieux utiliserla méthode de RandomUnderSampler

In [16]:
# nous avons vu dans notre analyse de données que les classes 2 et 3 étaient sous représentées
# afin de régler ce déséquilibre de classe, nous allons faire un SMOTE et l'appliquer à nos données

ru =RandomUnderSampler()
X_train_resampled, y_train_resampled = ru.fit_resample(X_train, y_train)
print('Classes échantillon undersampled :', dict(pd.Series(y_train_resampled).value_counts()))

Classes échantillon undersampled : {1: 1083, 2: 1083, 3: 1083}


## 2- Entrainement de plusieurs modèles

In [19]:
# entrainement d un arbre de decision

model_tree_clf = DecisionTreeClassifier(random_state=42, max_depth=10)

start_time = time.time()
model_tree_clf.fit(X_train_resampled, y_train_resampled)
end_time = time.time()

training_time = end_time - start_time
print(f"Temps d'entraînement du modèle : {training_time:.2f} secondes")

y_pred = model_tree_clf.predict(X_test)

print("Confusion Matrix:")
print(confusion_matrix(y_test, y_pred))

print("\nClassification Report:")
print(classification_report(y_test, y_pred))

print("\nAccuracy Score:")
print(accuracy_score(y_test, y_pred))

Temps d'entraînement du modèle : 0.05 secondes
Confusion Matrix:
[[4461 1177 2078]
 [  30  153   80]
 [ 514  633  874]]

Classification Report:
              precision    recall  f1-score   support

           1       0.89      0.58      0.70      7716
           2       0.08      0.58      0.14       263
           3       0.29      0.43      0.35      2021

    accuracy                           0.55     10000
   macro avg       0.42      0.53      0.39     10000
weighted avg       0.75      0.55      0.61     10000


Accuracy Score:
0.5488


In [20]:
# entrainement d un random forest

model_rf_clf = RandomForestClassifier(random_state=42)

start_time = time.time()
model_rf_clf.fit(X_train_resampled, y_train_resampled)
end_time = time.time()

training_time = end_time - start_time
print(f"Temps d'entraînement du modèle : {training_time:.2f} secondes")

y_pred = model_rf_clf.predict(X_test)

print("Confusion Matrix:")
print(confusion_matrix(y_test, y_pred))

print("\nClassification Report:")
print(classification_report(y_test, y_pred))

print("\nAccuracy Score:")
accuracy=accuracy_score(y_test, y_pred)
print(accuracy)

Temps d'entraînement du modèle : 0.64 secondes
Confusion Matrix:
[[5139 1047 1530]
 [  25  179   59]
 [ 551  705  765]]

Classification Report:
              precision    recall  f1-score   support

           1       0.90      0.67      0.77      7716
           2       0.09      0.68      0.16       263
           3       0.32      0.38      0.35      2021

    accuracy                           0.61     10000
   macro avg       0.44      0.58      0.43     10000
weighted avg       0.76      0.61      0.67     10000


Accuracy Score:
0.6083


In [22]:
# on sauvegarde l accuracy en memoire pour qu elle puisse etre communiqué aux admin par notre API
model_data = {
    'model': model_rf_clf,
    'accuracy': accuracy
}

# modifier seuil des tests unitaires pour déclencher le ré-entrainement

In [23]:
# nous souhaitons un modele predisant mieux les gravites eleves,
# on sauvegarde donc le modele avec la meilleure
import joblib
with open('../models/model_rf_clf.pkl', 'wb') as file:
    joblib.dump(model_data, file)